# Basic usage

This tutorial will introduce you to the definition of the building blocks for your
boson sampling experiment. The general workflow for a simple simulation is to define
an [`Input`](@ref) that enters into a [`Interferometer`](@ref) and ask what is the
probability to get a defined [`OutputMeasurement`](@ref).

They are linked together through an [`Event`](@ref) type, which holds all the information relating to the virtual experiment. 


## Input

`BosonSampling.jl` provides different types of inputs. Here we will restrict to Fock states although Gaussian inputs are also available. We sort between three distinct types of input depending on the
distinguishability of the particles we want to make interfere: [`Bosonic`](@ref),
[`PartDist`](@ref) and [`Distinguishable`](@ref). The type [`PartDist`](@ref) is a container for different models of partial distinguishability. Currently available models are:
* [`OneParameterInterpolation`](@ref)
* [`RandomGramMatrix`](@ref)
* [`UserDefinedGramMatrix`](@ref)

In order to define the input, we first need to provide a [`ModeOccupation`](@ref) that describes the repartition of the particles among the modes.

In [1]:
using BosonSampling#main
using Plots
using Distributions

In [2]:
n = 3; # photon number
m = 6; # mode number

my_mode_occupation = ModeOccupation(random_occupancy(n,m))

state = [0, 0, 1, 0, 2, 0]

In the example above, `my_mode_occupation` has been created thanks to [`random_occupancy`](@ref) that randomly places `n` particles among `m` modes.
Let's build an input made off indistinguishable photons by using the type [`Bosonic`](@ref)

In [ ]:
my_input = Input{Bosonic}(my_mode_occupation)


at creation, the Input took into account the partial distinguishability (here indistinguishable) of the photons, and created fields to hold this information. You can find all fields of a Type through

In [ ]:
fieldnames(Input)

In this case, the gram matrix G was generated:

In [ ]:
my_input.G

more info on the behaviour of a Type or Function is available just by doing

In [ ]:
?GramMatrix

You can also access all methods (functions) acting on a type (here `Subset`) with

In [ ]:
methodswith(Subset)
# this may take a minute

### Exercise

Build an input of distinguishable 10 photons in 20 modes, with all photon distinguishable. The input should consist of three photons in the first mode, and the rest randomly distributed.

## Interferometer

The second building block of our boson sampler is the interferometer
we want to apply on `my_input`. A common practice to study boson sampling is to
pick up at random a Haar distributed unitary matrix that will represent the interferometer.
This can be done as follow:

In [3]:
my_random_interf = RandHaar(m);

my_random_interf.U

6×6 Matrix{ComplexF64}:
  -0.54492+0.0756887im  -0.0927714-0.12793im   …  -0.223639-0.497694im
 -0.168567-0.160195im    -0.295389-0.214419im     -0.424557-0.081141im
  0.364372-0.0904026im   -0.027895+0.320015im     -0.191117-0.496002im
  0.277427+0.374341im   -0.0528706-0.192462im     -0.240666+0.0884809im
 0.0709919+0.411134im      0.23834+0.457145im       0.11788-0.308195im
 -0.028148-0.332198im    -0.188922+0.630313im  …  -0.241386-0.00370043im

where we have accessed to the matrix thanks to the field `.U`.
We may also need to use a specific interferometer such as a [`Discrete Fourier Transform`](https://en.wikipedia.org/wiki/Discrete_Fourier_transform) or the [`Hadamard transform`](https://en.wikipedia.org/wiki/Hadamard_transform):

In [ ]:
my_fourier_interf = Fourier(m)

In [ ]:
my_hadamard_tf = Hadamard(2^m)

### Exercise:

Find a method (from the package) to check that a matrix is unitary. Apply it to a `RandHaar` interferometer of your choice.

## OutputMeasurement

Now consider what you want to observe, in this numerical experiment. For the standard BosonSampling setup, as proposed by AA, we would use an [`OutputMeasurement`](@ref) type called [`FockDetection`](@ref), each detector reading the number of photons in its mode. 

### Exercise 
Various other detection types are available, including mode realistic detectors (with dark counts and inefficiency). We can also bin modes together in subsets, making a partition. Do you remember how you can find more about them? Try to print all available ones.

Similary to the definition of the [`Input`](@ref), it is also possible to define an output configuration from a [`ModeOccupation`](@ref)

In [4]:
n = 3;

m=n^2;

my_mode_occupation = first_modes(n,m);

my_input = Input{Bosonic}(my_mode_occupation)

out = FockDetection(my_mode_occupation)

FockDetection(state = [1, 1, 1, 0, 0, 0, 0, 0, 0])

using [`FockDetection`](@ref). 

## Event

A numerical simulation is packed into an [`Event`](@ref) type. It stores everything about the experiment, including input, output, interferometer, probabibilities. Let's see how it works

In [5]:
my_interf = Fourier(my_input.m)
ev = Event(my_input, out, my_interf)

Event:

input state: state = [1, 1, 1, 0, 0, 0, 0, 0, 0] (Bosonic)

output measurement: FockDetection(state = [1, 1, 1, 0, 0, 0, 0, 0, 0])

Interferometer :

Type : Fourier
m : 9
U : 
ComplexF64[0.3333333333333333 + 0.0im 0.3333333333333333 + 0.0im 0.3333333333333333 + 0.0im 0.3333333333333333 + 0.0im 0.3333333333333333 + 0.0im 0.3333333333333333 + 0.0im 0.3333333333333333 + 0.0im 0.3333333333333333 + 0.0im 0.3333333333333333 + 0.0im; 0.3333333333333333 + 0.0im 0.25534814770632597 + 0.21426253656217975im 0.057882725888976805 + 0.32826925100406934im -0.16666666666666657 + 0.28867513459481287im -0.3132308735953028 + 0.11400671444188962im -0.3132308735953028 - 0.11400671444188955im -0.1666666666666668 - 0.2886751345948128im 0.05788272588897665 - 0.32826925100406934im 0.2553481477063259 - 0.21426253656217986im; 0.3333333333333333 + 0.0im 0.057882725888976805 + 0.32826925100406934im -0.3132308735953028 + 0.11400671444188962im -0.1666666666666668 - 0.2886751345948128im 0.2553481477063259 - 0

, nothing, nothing)


### Exercise

Now, can you find the probability of `ev`? It is stored in it, do you remember how to see what available fields are?

You may have noticed that the probability is... empty. Indeed, as computing these probabilities is often the computational bottleneck, their evaluation is delayed. Julia allows to keep a field empty, as a `Nothing` type. Interestingly, this is efficient for the compiler. Now, we can compute the probability as

In [6]:
compute_probability!(ev)

0.015964548319225582

### Exercise 

Compute the probability that partially distinguishable photons populating the `n=3`
first modes of `m=9` modes end up in the `n` last output modes when interfering through
a random interferometer

## Using the BosonSampling types

Julia allows to define functions that act on new types, such as [`ModeOccupation`](@ref) defined in this package, through a syntax that would otherwise be reserved for core-objects such as `Float`, `Int`.

This allows to intuitively act on custom types. For instance, two [`ModeOccupation`](@ref) can see their state summed by simply using `+`


In [ ]:
n=3
m=4
s1 = ModeOccupation([1,2,3,4])
s2 = ModeOccupation([1,0,1,0])

s1+s2

Some functions of interest are [`zeros(mo::ModeOccupation)`](@ref), [`Base.cat(s1::ModeOccupation, s2::ModeOccupation)`](@ref) for instance.

# Sampling

Let's now use boson samplers with a realistic detector (see properties in Documentation)

In [ ]:
n = 10
m = 10

p_dark = 0.1
p_no_count = 0.1

input_state = first_modes(n,m)
interf = RandHaar(m)

i = Input{Bosonic}(input_state)
o = RealisticDetectorsFockSample(p_dark, p_no_count)
ev = Event(i,o,interf)

BosonSampling.sample!(ev)


this works intelligently for different types of inputs, detectors with the same `sample!` function

In [ ]:
i = Input{Distinguishable}(input_state)
o = FockSample()
ev = Event(i,o,interf)

BosonSampling.sample!(ev)

# Circuits

Let's now see how to build optical circuits from elementary optical elements such as beam splitters. We first define an empty circuit, then add elements as follows

In [ ]:
n = 3
m = n

circuit = LosslessCircuit(m)
interf = BeamSplitter(1/sqrt(2)) # the element we add
target_modes = ModeList([1, 2],m) # acting on the first two modes
add_element!(circuit, interf, target_modes)
circuit

Various other elements are available, for instance, we can now add a random phase shift on the last mode

In [ ]:
d = Uniform(0, 2pi)

interf = RandomPhaseShifter(d)
target_modes = ModeList([3],m) # acting on the last mode
add_element!(circuit, interf, target_modes)
circuit.U

# Loss

Loss can be incorporated through [`BeamSplitter`](@ref)'s sending photons with some probability to extra environment modes. If a physical [`Interferometer`](@ref) has `m` modes, we create extra `m` modes representing lost photons. In reality, these would not be accessible, but we may still keep this information if necessary. This allows to post-select events upon a certain loss pattern, such as finding `l` (lost) photons in the environment modes.

## Conversions

In general, the function [`to_lossy`](@ref) converts physical `m`-mode objects into their `2m`-modes counterpart fitting the above model. For instance

In [ ]:
n=3
m=4
first_modes(n,m)

In [ ]:
to_lossy(first_modes(n,m))

In [ ]:
# creating a Subset:
Subset(first_modes(n,m))

In [ ]:
# expanding it doesn't change the Subset
to_lossy(Subset(first_modes(n,m)))

In [ ]:
# but it is now of the correct size
to_lossy(Subset(first_modes(n,m))).m

## Conventions

Each circuit element, such as [`BeamSplitter`](@ref) and [`PhaseShift`](@ref) can bear a certain amount of loss. We write it `η_loss`. It is the transmission amplitude of the beam splitter representing the loss process. Therefore the probability that a photon is not lost is `η_loss^2`.

## Lossy interferometers

The inclusion of loss creates bigger [`Interferometer`](@ref)'s, but half of their modes are not accessible (environment). For this reason, we use the subtype [`LossyInterferometer`](@ref).

The fields are named in such a way that all computations can be done without changes, as if we now used a `2m*2m` lossless interferometer. The physical quantities are labelled accordingly such as `m_real` and `U_physical`.

## Models implemented

Let us now discuss the various lossy elements available.
* [`UniformLossInterferometer`](@ref) : This simplest model is one where photons have an identical chance of being lost.
* [`GeneralLossInterferometer`](@ref) This is a generic model as described in ...
* Lossy circuit elements : When constructing a [`Circuit`](@ref) from elements, each element has its own loss characteristics. We also introduce lines, representing for instance optical fibers that have no interaction but can still be lossy.
* [`LossyCircuit`](@ref): circuits with lossy elements.

## Circuits

When using `circuit_elements` to construct a lossy interferometer, the loss channel associated to mode `i` will always be mode `m+i`. We now give two detailed examples, the first includes a lossy line (such as an optical fiber), the second does HOM with a lossy beam splitter.

In [ ]:
### one d ex ##

n = 1
m = 1

function lossy_line_example(η_loss)

    circuit = LossyCircuit(1)
    interf = LossyLine(η_loss)
    target_modes = ModeList([1],m)

    add_element_lossy!(circuit, interf, target_modes)
    circuit

end

lossy_line_example(0.9)

transmission_amplitude_loss_array = 0:0.1:1
output_proba = []

i = Input{Bosonic}(to_lossy(first_modes(n,m)))
o = FockDetection(to_lossy(first_modes(n,m)))

for transmission in transmission_amplitude_loss_array

    ev = Event(i,o, lossy_line_example(transmission))
    compute_probability!(ev)
    push!(output_proba, ev.proba_params.probability)
end

print(output_proba)

plot(transmission_amplitude_loss_array, output_proba)
ylabel!("p no lost")
xlabel!("transmission amplitude")


In [ ]:
### 2d HOM with loss example ###

n = 2
m = 2
i = Input{Bosonic}(first_modes(n,m))
o = FockDetection(ModeOccupation([2,0])) # detecting bunching, should be 0.5 in probability if there was no loss
transmission_amplitude_loss_array = 0:0.1:1
output_proba = []

function lossy_bs_example(η_loss)

    circuit = LossyCircuit(2)
    interf = LossyBeamSplitter(1/sqrt(2), η_loss)
    target_modes = ModeList([1,2],m)

    add_element_lossy!(circuit, interf, target_modes)
    circuit

end

for transmission in transmission_amplitude_loss_array

    ev = Event(i,o, lossy_bs_example(transmission))
    compute_probability!(ev)
    push!(output_proba, ev.proba_params.probability)
end

plot(transmission_amplitude_loss_array, output_proba)
ylabel!("p bunching top mode")
xlabel!("transmission amplitude")

# Photon counting in partitions

One of the novel algorithmic tools in this package allows to bin the output modes of a BosonSampler, and to compute the probabilities associated with finding k_1 photons in the first bin, k_2 in the second, etc.

A brute force evaluation of these probabilities would be extremely costly, but our algorithm can do it efficiently. For instance:

In [ ]:
n = 20
m = 400

# Experiment parameters
input_state = first_modes(n,m)
interf = RandHaar(m)
i = Input{Bosonic}(input_state)

# Subset selection
s = Subset(first_modes(Int(m/2),m))
part = Partition(s)

# Want to find all photon counting probabilities
o = PartitionCountsAll(part)

# Define the event and compute probabilities
ev = Event(i,o,interf)
compute_probability!(ev)
